In [1]:
# Put in PaGraph/partition/
# Remove data import in both dg.py and ordering.py
# 
import ordering
from dg import dg
from ogb.nodeproppred import DglNodePropPredDataset
import numpy as np

In [2]:
ROOT_DIR = '/data/sandeep'
PAGRAPH_DIR = '/data/sandeep/pagraph'
dataset = DglNodePropPredDataset('ogbn-arxiv',root=ROOT_DIR)
graph = dataset[0][0]
graph = graph.add_self_loop()
split_idx = dataset.get_idx_split()

In [3]:
scipyCOO = graph.adj(scipy_fmt='coo')
scipyCSR = scipyCOO.tocsc()

In [4]:
adj, vmap = ordering.reordering(scipyCSR, depth=3)

=>99%


In [5]:
train_nids = split_idx['train']
train_nids = np.sort(vmap[train_nids])

In [6]:
p_v, p_trainv = dg(4, adj, train_nids, 3)

total vertices: 169343 | train vertices: 90941
=>99%
vertex# with self-reliance:  126904
vertex# w/o  self-reliance:  22702
vertex# with self-reliance:  126070
vertex# w/o  self-reliance:  22734
vertex# with self-reliance:  124972
vertex# w/o  self-reliance:  22776
vertex# with self-reliance:  126197
vertex# w/o  self-reliance:  22729


In [51]:
def get_neighs(nid,depth):
  neighs = [nid]
  new_neighs = []
  for _ in range(depth):
    for i in neighs:
      new_neighs.extend(graph.out_edges(i)[1].tolist())
    neighs = np.unique(new_neighs)
    new_neighs = []
  return neighs

print(get_neighs(371,3))
  

[   371   9137  38093  46585  49280  63174  74486  85597  92744  96778
 100567 111326 120060 126704 132526 141791 143095 145787 147852 151709
 158560]


In [44]:
res = dict()
for i in range(graph.number_of_nodes()):
  if i not in train_nids:
    continue
  res[i] = [j for j in range(4) if i in p_v[j]]
  if len(res[i]) == 0:
    print("NODE",i)

In [47]:
all_nodes = []
for i in p_v:
  all_nodes.extend(i.tolist())
all_nodes = np.unique(all_nodes)

cache_rate = [len(i)/len(all_nodes) for i in p_v]
cache_rate

[0.7654872060899254,
 0.7604565031185533,
 0.7538333474080419,
 0.7612225693983665]